In [2]:
import dataverk as dv
import pandas as pd
import ipywidgets as widgets

import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as plio

# Get data

In [3]:
url="https://www.nav.no/no/NAV+og+samfunn/Statistikk/Flere+statistikkomrader/Trygdemisbruk/_attachment/536372?_download=true&_ts=16140e9fd20"
df = pd.read_excel(url, skiprows=7)[1:20]
df.columns = ["Fylke", "Antall personer", "Anmeldt beløp"]

In [4]:
df.head(5)

,Fylke,Antall personer,Anmeldt beløp
1,Østfold,69.0,10639774.0
2,Akershus,108.0,22599182.0
3,Oslo,167.0,30772319.0
4,Hedmark,32.0,4669307.0
5,Oppland,21.0,3112811.0


# Create views

In [6]:
data = [ 
go.Bar(
    x=df['Fylke'], 
    y=df['Antall personer'],
    name='Antall personer'
),
go.Bar(
    x=df['Fylke'], 
    y=df['Anmeldt beløp'],
    name='Anmeldt beløp',
    yaxis='y2'
)
]

layout = go.Layout(
    yaxis=dict(
        title='Antall personer',
        side='left'
    ),
    yaxis2=dict(
        title='Anmeldt beløp',
        anchor='x',
        overlaying='y',
        side='right'
    ),
    barmode='group'
)

fig_1 = go.Figure(data,layout)

go.FigureWidget(data,layout)

FigureWidget({
    'data': [{'name': 'Antall personer',
              'type': 'bar',
              'uid': 'd0f…

# Create datapackage

In [9]:
readme = """
# Trygdemisbruk

"""

metadata = {
'store': 'https://raw.githubusercontent.com/datasett/nav-examples/master',
'title': 'Trygdemisbruk',
'readme': readme,
'license':'MIT',
'accessRights':'Open',
'auth': 'NAV',
'description':'Trygdemisbruk',
'name':'Trygdemisbruk',
'source':'Dataverk',
'keywords':['test'],
'provenance':'NAV',
'publisher': 'NAV',
'project': 'mesiqi', 
'user':'dataverk',
'bucket_name': 'dataparcel',
'dataset':'trygdemisbruk'
}

In [10]:
dp = dv.Datapackage(metadata)

## Add views to datapackage

In [11]:
description = '''
Trygdemisbruk 2017 

'''

dp.add_view(
    spec_type='plotly',
    name='Trygdemisbruk 2017',
    title='Trygdemisbruk 2017',
    description = description,
    resources=[],
    attribution = "Kilde: NAV.",
    spec=plio.to_json(fig_1)
)

# Save data and publish datapackage

In [16]:
client = dv.Client()
client.publish(dp)

2019-05-14T11:15:10.202789: B149899 <class 'dataverk.connectors.google_storage.GoogleStorageConnector'>: String (format: json) written to https://storage.googleapis.com/dataparcel/b129d80e1a7ba0795ed86184a40c4357/datapackage.json
2019-05-14T11:15:10.735262: B149899 <class 'dataverk.connectors.elasticsearch.ElasticsearchConnector'>: Document b129d80e1a7ba0795ed86184a40c4357 of type odata indexed to elastic index: odata.
